# Machine Learning with Sci-Kit Learn

- Based on [Keith Galli's sklearn tutorial](https://github.com/KeithGalli/sklearn)

## Data Class

- Create a Review class for each token of the data
- This also demonstrates how Object-Oriented language works with the class

In [1]:
import random

class Sentiment:
    NEGATIVE = 'NEGATIVE'
    NEUTRAL = 'NEUTRAL'
    POSITIVE = 'POSITIVE'

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else:
            return Sentiment.POSITIVE

class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews
    def get_text(self):
        return [x.text for x in self.reviews]
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)

## Data Loading

In [2]:
import json

fname = '../data/books_small_10000.json'

reviews = []
with open(fname) as f:
    for line in f:
        cur_review = json.loads(line)
        reviews.append(Review(cur_review['reviewText'], cur_review['overall']))

In [3]:
print('Number of Reviews: {}'.format(len(reviews)))
print('Sample Text of Doc 1:')
print('-'*30)
print(reviews[0].text)

Number of Reviews: 10000
Sample Text of Doc 1:
------------------------------
I bought both boxed sets, books 1-5.  Really a great series!  Start book 1 three weeks ago and just finished book 5.  Sloane Monroe is a great character and being able to follow her through both private life and her PI life gets a reader very involved!  Although clues may be right in front of the reader, there are twists and turns that keep one guessing until the last page!  These are books you won't be disappointed with.


In [4]:
## Check Sentiment Distribution of the Current Dataset
from collections import Counter
sentiment_distr = Counter([r.get_sentiment() for r in reviews])
print(sentiment_distr)

Counter({'POSITIVE': 8378, 'NEUTRAL': 978, 'NEGATIVE': 644})


## Splitting Data into Train and Test Sets

In [5]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(reviews, test_size = 0.33, random_state=42)

In [6]:
## Sentiment Distrubtion for Train and Test
print(Counter([r.get_sentiment() for r in train]))
print(Counter([r.get_sentiment() for r in test]))

Counter({'POSITIVE': 5611, 'NEUTRAL': 653, 'NEGATIVE': 436})
Counter({'POSITIVE': 2767, 'NEUTRAL': 325, 'NEGATIVE': 208})


## Balance the Classes

In [7]:
train_container = ReviewContainer(train)
test_container = ReviewContainer(test)

# balance
train_container.evenly_distribute()
test_container.evenly_distribute()

# check sentiment distribution again
print(Counter([r.get_sentiment() for r in train_container.reviews]))
print(Counter([r.get_sentiment() for r in test_container.reviews]))



Counter({'POSITIVE': 436, 'NEGATIVE': 436})
Counter({'NEGATIVE': 208, 'POSITIVE': 208})


## Train and Test Data and Labels

In [8]:
train_text = train_container.get_text()
train_label = train_container.get_sentiment()
test_text = test_container.get_text()
test_label = test_container.get_sentiment()

# print(train_label.count(Sentiment.POSITIVE))
# print(train_label.count(Sentiment.NEGATIVE))
print(Counter(train_label))

Counter({'POSITIVE': 436, 'NEGATIVE': 436})


## Vectorization: Bag-of-Words Model

:::{admonition,important}

- Always split the data into train and test first before vectorizing the texts
- Otherwise, you would leak information to the training process, which may lead to over-fitting
- When vectorizing the texts, `fit_transform()` the train and `transform()` the test
:::

In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

tfidf_vec = TfidfVectorizer()
train_text_bow = tfidf_vec.fit_transform(train_text) # fit train
test_text_bow = tfidf_vec.transform(test_text) # transform test

In [10]:
print(train_text_bow.shape)
print(test_text_bow.shape)
print(type(train_text_bow))
print(train_text_bow[0])

(872, 8906)
(416, 8906)
<class 'scipy.sparse.csr.csr_matrix'>
  (0, 7961)	0.18898521177750435
  (0, 5361)	0.10060299835684441
  (0, 7938)	0.08597057225949589
  (0, 8572)	0.11293099066195995
  (0, 1219)	0.07272247449484075
  (0, 8782)	0.16120861282120072
  (0, 1362)	0.07256729873660205
  (0, 8840)	0.13986216847842725
  (0, 3090)	0.20046778294278353
  (0, 370)	0.1325605757227514
  (0, 5408)	0.05232537619760367
  (0, 7063)	0.08076594346687482
  (0, 7683)	0.11634711556792275
  (0, 3177)	0.04890580965524824
  (0, 4597)	0.20046778294278353
  (0, 6411)	0.07532840415925975
  (0, 8704)	0.07654850999320897
  (0, 1283)	0.13130172380939384
  (0, 4034)	0.04646406864417813
  (0, 8872)	0.10931081285551784
  (0, 8518)	0.3157461111028675
  (0, 5478)	0.041198390932991
  (0, 5092)	0.12284855012097678
  (0, 4212)	0.16236482856346054
  (0, 3374)	0.07782355836398366
  :	:
  (0, 8590)	0.1170827284855665
  (0, 7505)	0.1061014005828395
  (0, 423)	0.07162896829742499
  (0, 1357)	0.09330140560116854
  (0, 3660)	

## Classification Models

### Support Vector Machine (SVM)

In [11]:
from sklearn import svm

model_svm = svm.SVC(kernel='linear')
model_svm.fit(train_text_bow, train_label)

SVC(kernel='linear')

In [12]:
model_svm.predict(test_text_bow[:10])
#print(model_svm.score(test_text_bow, test_label))

array(['NEGATIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE',
       'NEGATIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'NEGATIVE'],
      dtype='<U8')

### Decision Tree

In [13]:
from sklearn.tree import DecisionTreeClassifier

model_dec = DecisionTreeClassifier()
model_dec.fit(train_text_bow, train_label)

model_dec.predict(test_text_bow[:10])

array(['NEGATIVE', 'NEGATIVE', 'NEGATIVE', 'NEGATIVE', 'NEGATIVE',
       'NEGATIVE', 'NEGATIVE', 'POSITIVE', 'NEGATIVE', 'NEGATIVE'],
      dtype='<U8')

### Naive Bayes

In [14]:
from sklearn.naive_bayes import GaussianNB
model_gnb = GaussianNB()
model_gnb.fit(train_text_bow.toarray(), train_label)

model_gnb.predict(test_text_bow[:10].toarray())

array(['NEGATIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE',
       'POSITIVE', 'NEGATIVE', 'NEGATIVE', 'POSITIVE', 'NEGATIVE'],
      dtype='<U8')

### Logistic Regression

In [15]:
from sklearn.linear_model import LogisticRegression

model_lg = LogisticRegression()
model_lg.fit(train_text_bow, train_label)

model_lg.predict(test_text_bow[:10].toarray())

array(['NEGATIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE',
       'NEGATIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'NEGATIVE'],
      dtype='<U8')

## Evaluation

In [16]:
#Mean Accuracy
print(model_svm.score(test_text_bow, test_label))
print(model_dec.score(test_text_bow, test_label))
print(model_gnb.score(test_text_bow.toarray(), test_label))
print(model_lg.score(test_text_bow, test_label))

0.8076923076923077
0.6370192307692307
0.6610576923076923
0.8052884615384616


In [17]:
# F1
from sklearn.metrics import f1_score

f1_score(test_label, model_svm.predict(test_text_bow), average=None, labels = [Sentiment.POSITIVE, Sentiment.NEGATIVE])

array([0.80582524, 0.80952381])

In [18]:
## try a whole new self-created review:)
new_review =['This book looks soso like the content but the cover is weird',
             'This book looks soso like the content and the cover is weird'
            ]
new_review_bow = tfidf_vec.transform(new_review)
model_svm.predict(new_review_bow)

array(['NEGATIVE', 'POSITIVE'], dtype='<U8')

## Tuning Model

In [19]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel': ('linear', 'rbf'), 'C': (1,4,8,16,32)}

svc = svm.SVC()
clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(train_text_bow, train_label)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': (1, 4, 8, 16, 32), 'kernel': ('linear', 'rbf')})

In [20]:
sorted(clf.cv_results_.keys())
print(clf.best_params_)

{'C': 1, 'kernel': 'linear'}


In [21]:
print(clf.score(test_text_bow, test_label))

0.8076923076923077


## Saving Model


In [22]:
#  import pickle

# with open('../ml-sent-svm.pkl', 'wb') as f:
#     pickle.dump(clf, f)
# with open('../ml-sent-svm.pkl' 'rb') as f:
#     loaded_svm = pickle.load(f)

In [23]:
# import pkg_resources
# import types
# def get_imports():
#     for name, val in globals().items():
#         if isinstance(val, types.ModuleType):
#             # Split ensures you get root package, 
#             # not just imported function
#             name = val.__name__.split(".")[0]

#         elif isinstance(val, type):
#             name = val.__module__.split(".")[0]

#         # Some packages are weird and have different
#         # imported names vs. system/pip names. Unfortunately,
#         # there is no systematic way to get pip names from
#         # a package's imported name. You'll have to add
#         # exceptions to this list manually!
#         poorly_named_packages = {
#             "PIL": "Pillow",
#             "sklearn": "scikit-learn"
#         }
#         if name in poorly_named_packages.keys():
#             name = poorly_named_packages[name]

#         yield name

In [24]:
# get_imports()

In [25]:
# imports = list(set(get_imports()))

# # The only way I found to get the version of the root package
# # from only the name of the package is to cross-check the names 
# # of installed packages vs. imported packages
# requirements = []
# for m in pkg_resources.working_set:
#     if m.project_name in imports and m.project_name!="pip":
#         requirements.append((m.project_name, m.version))

# for r in requirements:
#     print("{}=={}".format(*r))